*CLEANING:*<br><br>
This second step updates our company attributes dataset to have most up to date tickers.
<br>Moreover investigation and cleaning of the columns (183 of them) were done to reduce them to most relvant ones (76)

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import yahoo_fin as yfin
import yfinance as yf


In [2]:
df = pd.read_pickle(r'C:\Users\49176\Desktop\DSA\OPA_repo\df_allInfo.pkl')


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 407 entries, 0 to 0
Columns: 148 entries, zip to stockPosition
dtypes: float64(84), object(64)
memory usage: 473.8+ KB


In [8]:
df = df.dropna(subset='symbol', axis=0)


In [9]:
cols_by_null_values = (df.isna().sum()*100/len(df)).sort_values(ascending=False)
df = df.drop(list(cols_by_null_values[cols_by_null_values.values == 100].index), axis=1)


In [10]:
rem_cols_by_null_value = (df.isna().sum()*100/len(df)).sort_values(ascending=False)
rem_cols_gt_99 = rem_cols_by_null_value[rem_cols_by_null_value.values > 99].index
for col in list(rem_cols_gt_99):
    print(df[col].value_counts())


0.8541    1
Name: stockPosition, dtype: int64
[{'realestate': 0}, {'consumer_cyclical': 0.3168}, {'basic_materials': 0}, {'consumer_defensive': 0}, {'technology': 0.4097}, {'communication_services': 0.0669}, {'financial_services': 0}, {'utilities': 0}, {'industrials': 0.060700003}, {'energy': 0}, {'healthcare': 0}]    1
Name: sectorWeightings, dtype: int64
Technology    1
Name: category, dtype: int64
Clockwise Capital Innovation ETF    1
Name: fundFamily, dtype: int64
17.21    1
Name: navPrice, dtype: int64
4282143    1
Name: totalAssets, dtype: int64
True    1
Name: err, dtype: int64
0.0    1
Name: bondPosition, dtype: int64
[]    1
Name: holdings, dtype: int64
{'maturity': None, 'duration': None, 'creditQuality': None, 'maturityCat': None, 'durationCat': None, 'creditQualityCat': None}    1
Name: bondHoldings, dtype: int64
Exchange Traded Fund    1
Name: legalType, dtype: int64
1643241600    1
Name: fundInceptionDate, dtype: int64
[{'bb': 0}, {'aa': 0}, {'aaa': 0}, {'a': 0}, {'other'

In [11]:
#not very useful data for these >99% null columns, hence we drop them
df = df.drop(list(rem_cols_gt_99), axis=1)
rem_cols_by_null_value = (df.isna().sum()*100/len(df)).sort_values(ascending=False)

!!IMPORTANT!!<br> On discovery of data incompletness this step was added to update our tickers.<br>
Kindly note that we didnot directly created a new company data attributes dataset for all 503 stocks as this is a time expensive step and hence reused the already downloaded data and only decided to add/remove missing/outdated ticker informations.

In [12]:
from yahoo_fin import stock_info as si
sp500_tickers = si.tickers_sp500(True)

add the missing tickers into our df_allInfo dataframe

In [14]:
sp_data_ticker = list(sp500_tickers.Symbol.values)
df_allInfo = pd.DataFrame([])
issue_tickers = []
for t in list(set(sp_data_ticker) - set(df.symbol)):
    try:
        temp_df = pd.DataFrame.from_dict([yf.Ticker(t).info]) 
        df_allInfo = pd.concat([df_allInfo, temp_df])
    except:
        issue_tickers.append(t)
        continue

Check if there are any issue tickers

In [15]:
len(issue_tickers)

0

In [19]:
df_allInfo.info()
#df_allInfo.select_dtypes(include='bool').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161 entries, 0 to 0
Columns: 156 entries, zip to address2
dtypes: bool(2), float64(68), int64(9), object(77)
memory usage: 195.3+ KB


do an inner join to only keep currently valid tickers in our original company attributes dataset OR effectively removing the outdated tickers

In [20]:
df = df.merge(sp500_tickers.Symbol, left_on='symbol', right_on='Symbol', how='inner').drop('Symbol', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 342 entries, 0 to 341
Columns: 127 entries, zip to uuid
dtypes: float64(79), object(48)
memory usage: 342.0+ KB


finally append this with the missing ones

In [21]:
df = pd.concat([df, df_allInfo]) 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 503 entries, 0 to 0
Columns: 158 entries, zip to fromCurrency
dtypes: float64(72), object(86)
memory usage: 624.8+ KB


C:\Users\49176\AppData\Local\Temp\ipykernel_22532\1318890022.py:1: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, df_allInfo])


In [22]:
cols_by_null_values = (df.isna().sum()*100/len(df)).sort_values(ascending=False)
df = df.drop(list(cols_by_null_values[cols_by_null_values.values == 100].index), axis=1)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 503 entries, 0 to 0
Columns: 127 entries, zip to uuid
dtypes: float64(72), object(55)
memory usage: 503.0+ KB


In [24]:
rem_cols_by_null_value = (df.isna().sum()*100/len(df)).sort_values(ascending=False)

check each column and see if they are an interesting feature -> features_for_viz.txt

In [25]:
col = 'fax'
display(rem_cols_by_null_value[col])
display(df[col].describe())
display(df[col].value_counts())

85.48707753479125

count               73
unique              73
top       866 497 1134
freq                 1
Name: fax, dtype: object

866 497 1134    1
415-394-9001    1
305 406 8630    1
330 684 3370    1
847 913 8766    1
               ..
508 872 1990    1
404-439-4911    1
305 436 4140    1
269 385 1062    1
440 892 9507    1
Name: fax, Length: 73, dtype: int64

cols_keep list contains the columns considered relevant and will be kept for our visualisation step.

In [26]:
cols_keep = [#'fax',
             #'address2',
             #'uuid',
             'trailingPegRatio',
             #'lastDividendDate',
             'lastDividendValue',
             #'preMarketPrice',
             #'impliedSharesOutstanding',
             '52WeekChange',
             'SandP52WeekChange',
             'fiveYearAvgDividendYield',
             #'lastSplitDate',
             'lastSplitFactor',
             'dividendRate',
             'dividendYield',
             #'exDividendDate',
             'earningsGrowth',
             'earningsQuarterlyGrowth',
             'debtToEquity',
             'freeCashflow',
             'trailingPE',
             'trailingAnnualDividendYield',
             'trailingAnnualDividendRate',
             'ebitda',
             'enterpriseToEbitda',
             'priceToBook',
             'returnOnEquity',
             'currentRatio',
             'quickRatio',
             'operatingCashflow',
             'state',
             'beta',
             'targetMeanPrice',
             'targetMedianPrice',
             'targetHighPrice',
             'numberOfAnalystOpinions',
             'targetLowPrice',
             'recommendationMean',
             'payoutRatio',
             'priceToSalesTrailing12Months',
             'returnOnAssets',
             'pegRatio',
             'totalCashPerShare',
             'totalRevenue',
             'revenueGrowth',
             'enterpriseToRevenue',
             'totalDebt',
             'floatShares',
             'totalCash',
             'netIncomeToCommon',
             'marketCap',
             'fullTimeEmployees',
             'forwardPE',
             'trailingEps',
             'forwardEps',
             'sharesOutstanding',
             'bookValue',
             'shortPercentOfFloat',
             #'phone',
             'twoHundredDayAverage',
             #'previousClose',
             #'regularMarketOpen',
             #'tradeable',
             #'dayHigh',
             #'regularMarketPrice',
             #'ask',
             #'bidSize',
             #'averageVolume',
             #'bid',
             'fiftyTwoWeekLow',
             #'regularMarketDayHigh',
             #'regularMarketPreviousClose',
             'fiftyDayAverage',
             'fiftyTwoWeekHigh',
             #'open',
             #'averageVolume10days',
             #'volume',
             #'regularMarketDayLow',
             'currency',
             #'askSize',
             #'regularMarketVolume',
             #'dayLow',
             #'averageDailyVolume10Day',
             #'zip',
             #'dateShortInterest',
             'ebitdaMargins',
             'revenuePerShare',
             #'financialCurrency',
             #'currentPrice',
             'grossProfits',
             'recommendationKey',
             'operatingMargins',
             'grossMargins',
             'profitMargins',
             'industry',
             'shortName',
             #'address1',
             #'maxAge',
             #'website',
             #'companyOfficers',
             'country',
             #'logo_url',
             'city',
             #'longBusinessSummary',
             'exchange',
             #'longName',
             #'priceHint',
             #'lastFiscalYearEnd',
             'enterpriseValue',
             'sharesShortPriorMonth',
             #'sharesShortPreviousMonthDate',
             'shortRatio',
             #'mostRecentQuarter',
             #'nextFiscalYearEnd',
             'heldPercentInsiders',
             'sector',
             'sharesPercentSharesOut',
             #'exchangeTimezoneName',
             'sharesShort',
             #'market',
             #'messageBoardId',
             'symbol',
             #'quoteType',
             #'gmtOffSetMilliseconds',
             #'isEsgPopulated',
             #'exchangeTimezoneShortName',
             'heldPercentInstitutions']


filter our df for the interesting columns and sort them from left to right by % availability <br>
save the dataset for the viz step

In [27]:
df = df[rem_cols_by_null_value[cols_keep].sort_values(ascending=True).index].set_index('symbol')
df.to_pickle('df_allInfo_clean.pkl')

In [28]:
df.head()

,exchange,shortName,twoHundredDayAverage,fiftyTwoWeekLow,fiftyDayAverage,fiftyTwoWeekHigh,currency,ebitdaMargins,grossProfits,recommendationKey,...,earningsQuarterlyGrowth,earningsGrowth,dividendYield,dividendRate,lastSplitFactor,fiveYearAvgDividendYield,SandP52WeekChange,52WeekChange,lastDividendValue,trailingPegRatio
symbol,,,,,,,,,,,,,,,,,,,,,
A,NYQ,"Agilent Technologies, Inc.",130.00755,112.52,138.2376,162.62,USD,0.28256,3.722000e+09,buy,...,-0.167,-0.151,0.0058,0.9,1398:1000,0.72,-0.16165,-0.031732,0.21,2.3275
AAL,NMS,"American Airlines Group, Inc.",15.04710,11.65,13.6508,21.42,USD,0.05309,1.934000e+09,hold,...,1.858,1.735,NaN,NaN,None,NaN,-0.16165,-0.213933,0.1,NaN
AAP,NYQ,Advance Auto Parts Inc.,187.62320,143.72,167.8564,244.55,USD,0.09143,4.931356e+09,buy,...,-0.346,-0.313,0.0392,6.0,3:2,0.79,-0.16165,-0.378157,1.5,0.9405
AAPL,NMS,Apple Inc.,153.73476,129.04,145.5186,182.94,USD,0.33105,1.707820e+11,buy,...,0.008,0.048,0.0063,0.92,4:1,0.99,-0.16165,-0.183745,0.23,2.8596
ABBV,NYQ,AbbVie Inc.,149.26730,120.75,149.0036,175.91,USD,0.50850,3.888300e+10,buy,...,0.242,0.242,0.0362,5.92,None,4.35,-0.16165,0.343399,1.41,NaN
